In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
import pandas as pd

from src import fetch_bucketed_from_bitmex, preprocess, evaluate

input_dir = 'cache_input'
model_dir = 'cache_model'
output_dir = 'cache_output'

# Crawling the Dataset

In [2]:
# df = fetch_bucketed_from_bitmex('XBTZ19', '1m')
df = pd.read_csv('cache_crawler/XBTZ19_1m.csv', index_col="timestamp", parse_dates=True)
df.describe()

,close,foreignNotional,high,homeNotional,lastSize,low,open,trades,turnover,volume,vwap
count,282449.000000,2.824510e+05,282449.000000,282451.000000,2.439640e+05,282449.000000,282449.000000,282451.000000,2.824510e+05,2.824510e+05,243964.000000
mean,9499.875292,3.691479e+04,9502.965474,3.947867,1.869157e+03,9496.680638,9499.880114,23.434890,3.947867e+08,3.691479e+04,9558.337851
std,1621.172019,1.371855e+05,1622.742756,16.638510,8.425148e+04,1619.518913,1621.167020,46.274287,1.663851e+09,1.371855e+05,1632.991936
min,6438.000000,0.000000e+00,6443.500000,0.000000,1.000000e+00,6431.000000,6438.000000,0.000000,0.000000e+00,0.000000e+00,6432.522800
25%,8158.000000,6.050000e+02,8159.500000,0.067004,5.000000e+01,8157.000000,8158.000000,2.000000,6.700365e+06,6.050000e+02,8180.628300
50%,9542.500000,7.336000e+03,9546.000000,0.779511,3.400000e+02,9540.000000,9542.500000,10.000000,7.795113e+07,7.336000e+03,9589.566600
75%,10616.500000,3.208350e+04,10619.500000,3.365414,1.321000e+03,10614.000000,10616.500000,26.000000,3.365414e+08,3.208350e+04,10656.436500
max,14600.000000,4.193417e+07,14600.000000,5874.536073,4.154957e+07,14539.000000,14600.000000,1529.000000,5.874536e+11,4.193417e+07,14547.570600


# Feature Engineering

In [3]:
preprocess(df, input_dir)

# Training

In [4]:
from src.trainer_linear import train

train(
    input_dir=input_dir,
    output_dir=output_dir,
    model_dir=model_dir,
    hidden_dim=64,
    lr=1e-3,
    batch_size=1024,
    n_epochs=4,
)

Using device cpu.
| epoch | trai loss | vali loss | time/s |
| ----: | --------- | --------- | -----: |
|     0 | 3.676e-01 | 3.535e-02 |      0 |
|     1 | 1.333e-01 | 2.267e-03 |      0 |
|     2 | 3.048e-02 | 1.935e-03 |      0 |
|     3 | 1.713e-02 | 4.360e-03 |      0 |


In [5]:
from src.trainer_lstm import train

train(
    input_dir=input_dir,
    output_dir=output_dir,
    model_dir=model_dir,
    hidden_dim=64,
    lstm_layers=1,
    dropout=0,
    lr=1e-3,
    batch_size=1024,
    n_epochs=4,
)

Using device cpu.
| epoch | trai loss | vali loss | time/s |
| ----: | --------- | --------- | -----: |
|     0 | 2.189e-01 | 2.414e-02 |     40 |
|     1 | 1.904e-02 | 5.068e-03 |     40 |
|     2 | 6.699e-03 | 5.035e-03 |     40 |
|     3 | 2.380e-03 | 4.607e-03 |     41 |


In [6]:
from src.trainer_lstmratio import train

train(
    input_dir=input_dir,
    output_dir=output_dir,
    model_dir=model_dir,
    hidden_dim=64,
    lstm_layers=1,
    dropout=0,
    lr=1e-3,
    batch_size=1024,
    n_epochs=4,
)

Using device cpu.
| epoch | trai loss | vali loss | time/s |
| ----: | --------- | --------- | -----: |
|     0 | 1.212e+00 | 6.064e-01 |     39 |
|     1 | 1.183e+00 | 5.995e-01 |     40 |
|     2 | 1.166e+00 | 5.955e-01 |     45 |
|     3 | 1.156e+00 | 5.944e-01 |     45 |


# Evaluation

In [7]:
evaluate(input_dir, output_dir)